# Manage with colab

In [0]:
from google.colab import files
uploaded = files.upload()

In [3]:
#colab
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse

# Colab Auth 
from google.colab import auth
auth.authenticate_user()

# Drive FUSE 
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

# drive/ を作り、そこにGoogle Driveをマウントする
!mkdir -p drive
!google-drive-ocamlfuse drive

gpg: keybox '/tmp/tmpo15uqf7j/pubring.gpg' created
gpg: /tmp/tmpo15uqf7j/trustdb.gpg: trustdb created
gpg: key AD5F235DF639B041: public key "Launchpad PPA for Alessandro Strada" imported
gpg: Total number processed: 1
gpg:               imported: 1
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the verification code: Access token retrieved correctly.


In [0]:
!wc -l test.json

648060 test.json


In [4]:
!ls 

datalab  drive


In [14]:
from os import path
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())

accelerator = 'cu80' if path.exists('/opt/bin/nvidia-smi') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.0-{platform}-linux_x86_64.whl torchvision
import torch
print(torch.__version__)
print(torch.cuda.is_available())

tcmalloc: large alloc 1073750016 bytes == 0x5c17a000 @  0x7f8028ea81c4 0x46d6a4 0x5fcbcc 0x4c494d 0x54f3c4 0x553aaf 0x54e4c8 0x54f4f6 0x553aaf 0x54efc1 0x54f24d 0x553aaf 0x54efc1 0x54f24d 0x553aaf 0x54efc1 0x54f24d 0x551ee0 0x54e4c8 0x54f4f6 0x553aaf 0x54efc1 0x54f24d 0x551ee0 0x54efc1 0x54f24d 0x551ee0 0x54e4c8 0x54f4f6 0x553aaf 0x54e4c8
0.4.0
True


In [15]:
!mkdir doc ; mkdir models ; mkdir data


mkdir: cannot create directory ‘doc’: File exists
mkdir: cannot create directory ‘models’: File exists
mkdir: cannot create directory ‘data’: File exists


In [0]:
!rm -f models/*

In [10]:
!wc -l big3.json

412872 big3.json


In [0]:
cp drive/colab/test_3/* .

In [0]:
!mv wordsOccurences.json doc/wordsOccurences.json

In [0]:
!head -n 200000 big3.json > test.json

In [12]:
!wc -l test.json

412872 test.json


# Import libraries

In [0]:
import numpy as np
import sys
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import TensorDataset, DataLoader
import json

# Process functions

In [0]:
def read_data(filePath):
    texts = []
    hashtags_by_text = []
    with open(filePath, 'r', encoding='utf8') as file:
        for line in file.readlines():
            json_obj = json.loads(line)
            texts.append(json_obj['text'])
            hashtags_by_text.append(json_obj['hashtags'])

    return hashtags_by_text, texts


def getFeatures(features, authorized_words):
    X = np.zeros((len(features), input_seq_size))
    vocab = set()
    vocab.add('<eos>')
    vocab.add('<UNK>')
    for  text in features:
        words = text.split()
        for word in words:
            if word in authorized_words:
                vocab.add(word)

    vocab_to_int = {word: i for i, word in enumerate(vocab)}
    rev_vocab = {vocab_to_int[key]: key for key in vocab_to_int}

    for j, text in enumerate(features):
        words = text.split()

        text_int = np.zeros(input_seq_size)
        for i in range(input_seq_size):
            if i <= len(words) - 1:
                if words[i] in authorized_words:
                    text_int[i] = vocab_to_int[words[i]]
                else:
                    text_int[i] = vocab_to_int['<UNK>']
            else:
                text_int[i] = vocab_to_int['<eos>']
        X[j] = text_int
        # texts_in_word_int.append(np.array([vocab_to_int[word] for word in text.split()]))
    return vocab_to_int, rev_vocab, X


def getLabels(labels):
    hashtags_set = set()
    for hashtags_in_text in labels:
        for hashtag in hashtags_in_text:
            hashtags_set.add(hashtag)

    hashtags_vocab_to_int = {hashtag: i for i, hashtag in enumerate(hashtags_set)}
    rev_hashtags_vocab_to_int = {hashtags_vocab_to_int[key]: key for key in hashtags_vocab_to_int}
    Y = np.zeros((len(labels), hashtags_seq_size))

    for j, hashtags in enumerate(labels):
        Y[j] = hashtags_vocab_to_int[hashtags[0]]
        # hashtags_seq = np.zeros(hashtags_seq_size)
        #
        # for i in range(hashtags_seq_size):
        #     if i <= len(hashtags) - 1:
        #         hashtags_seq[i] = hashtags_vocab_to_int[hashtags[i]]
        #     else:
        #         hashtags_seq[i] = -1
        # Y[j] = hashtags_seq
    return Y, hashtags_vocab_to_int, rev_hashtags_vocab_to_int


# Process Data

In [0]:
input_seq_size = 15
embed_size = 240
hashtags_seq_size = 1
hidden_size = 240
batch_size = 100
hashtags_by_text, texts = read_data('test.json')


with open('doc/wordsOccurences.json') as f:
    authorized_words = json.loads(f.read())

vocab_to_int, rev_vocab, X = getFeatures(texts, authorized_words.keys())

with open('doc/vocab.json', 'w') as f:
    f.write(json.dumps(vocab_to_int))

Y, hashtags_vocab_to_int, rev_hashtags_vocab_to_int = getLabels(hashtags_by_text)

with open('doc/vocabH.json', 'w') as f:
    f.write(json.dumps(hashtags_vocab_to_int))

nb_examples = X.shape[0] - 1

test_size = int(round(nb_examples / 10, 0))
train_size = int(nb_examples - test_size)

Y = Y.reshape((Y.shape[0]))

X = torch.LongTensor(X).cuda()
Y = torch.LongTensor(Y).cuda()

X_train = X[:train_size]
Y_train = Y[:train_size]
X_valid = X[train_size:]
Y_valid = Y[train_size:]

train_set = TensorDataset(X_train, Y_train)
valid_set = TensorDataset(X_valid, Y_valid)

train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(valid_set, batch_size=batch_size)



# Load data

In [0]:
vocabPath = 'doc/vocab.json'
vocabHPath = 'doc/vocabH.json'
seqLen = 15

with open(vocabPath, 'r') as f:
    vocab = json.loads(f.read())
with open(vocabHPath, 'r') as f:
    vocabH = json.loads(f.read())

rev_vocab = {vocab[key]: key  for key in vocab }
rev_vocabh = {vocabH[key]: key  for key in vocabH }

# Train

In [0]:
def perf(model, loader):
    criterion = nn.CrossEntropyLoss()
    model.eval()
    total_loss = correct = num = 0
    for x, y in loader:
        with torch.no_grad():
            y_scores = model(Variable(x).cuda())
            loss = criterion(y_scores, Variable(y).cuda())
        y_pred = torch.max(y_scores, 1)[1]
        correct += torch.sum(y_pred.data == y).item()
        total_loss += loss.data.item()
        num += len(y)
    return total_loss / num, correct


def train(model, epochs, quiet=False):
    # Selection de la fonction de coût
    criterion = nn.CrossEntropyLoss()
    # Selection de l'optimisateur ici SGD (Gradient Descent)
    optim = torch.optim.SGD(model.parameters(), lr=0.001)
    # Pour le nombre d'epoch spécifié faire
    lastAcc = 0
    for epoch in range(epochs):
        # Met le modèle en mode entrainement
        model.train()
        total_loss = correct = num = 0
        line = ''
        cpt = 0
        for X, y in train_loader:
            # Fait une prédiction (forward)
            y_scores = model(Variable(X).cuda())
            # Compare la prédiction a la valeur attendu calcul l'erreur (loss)
            loss = criterion(y_scores, Variable(y).cuda())
            # Calcul les gradients
            loss.backward()
            
            # Change les paramètres avec les gradient calculés
            optim.step()
            y_pred = torch.max(y_scores, 1)[1]
            correct += torch.sum(y_pred.data == y).item()
            total_loss += loss.data.item()
            num += len(y)
            cpt += 1
            if not quiet and cpt % 100 == 0:
                print('Epoch: ', epoch, ' ', num, ' / ', len(X_train), line)

        sys.stdout.write('\n')
        print('Epoch: ', epoch)
        print('Total loss train:', total_loss / num, '\nCorrect train: ', correct / len(X_train))
        loss_valid, correct_count_test = perf(model, valid_loader)
#         if correct_count_test > lastAcc:
#             lastAcc = correct_count_test
        torch.save(model, 'models/mod' + str(epoch) + '.mod')
        print('Total loss valid:', loss_valid, '\n', 'Correct test:', correct_count_test / len(X_valid))


class RNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.embed = nn.Embedding(len(vocab_to_int), embed_size)
        self.rnn = nn.GRU(embed_size, hidden_size, num_layers=1, dropout=0.0, bidirectional=False, batch_first=True)
        self.activation = nn.Tanh()
        self.decision = nn.Linear(hidden_size * 1 * 1, len(rev_hashtags_vocab_to_int))

    def forward(self, x):
        embed = self.embed(x)
        output, hidden = self.rnn(embed)
        postActivation = self.activation(hidden)
        cur = self.decision(postActivation.transpose(0, 1).contiguous().view(x.size(0), -1))
        return cur



model = RNN().cuda()

train(model, 10)


Epoch:  0   10000  /  371584 
Epoch:  0   20000  /  371584 
Epoch:  0   30000  /  371584 
Epoch:  0   40000  /  371584 
Epoch:  0   50000  /  371584 
Epoch:  0   60000  /  371584 
Epoch:  0   70000  /  371584 
Epoch:  0   80000  /  371584 
Epoch:  0   90000  /  371584 
Epoch:  0   100000  /  371584 
Epoch:  0   110000  /  371584 
Epoch:  0   120000  /  371584 
Epoch:  0   130000  /  371584 
Epoch:  0   140000  /  371584 
Epoch:  0   150000  /  371584 
Epoch:  0   160000  /  371584 
Epoch:  0   170000  /  371584 
Epoch:  0   180000  /  371584 
Epoch:  0   190000  /  371584 
Epoch:  0   200000  /  371584 
Epoch:  0   210000  /  371584 
Epoch:  0   220000  /  371584 
Epoch:  0   230000  /  371584 
Epoch:  0   240000  /  371584 
Epoch:  0   250000  /  371584 
Epoch:  0   260000  /  371584 
Epoch:  0   270000  /  371584 
Epoch:  0   280000  /  371584 
Epoch:  0   290000  /  371584 
Epoch:  0   300000  /  371584 
Epoch:  0   310000  /  371584 
Epoch:  0   320000  /  371584 
Epoch:  0   33000

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:193: UserWarning: Couldn't retrieve source code for container of type RNN. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


Total loss valid: 0.08409502820505742 
 Correct test: 0.2708777368726991
Epoch:  1   10000  /  371584 
Epoch:  1   20000  /  371584 
Epoch:  1   30000  /  371584 
Epoch:  1   40000  /  371584 
Epoch:  1   50000  /  371584 
Epoch:  1   60000  /  371584 
Epoch:  1   70000  /  371584 
Epoch:  1   80000  /  371584 
Epoch:  1   90000  /  371584 
Epoch:  1   100000  /  371584 
Epoch:  1   110000  /  371584 
Epoch:  1   120000  /  371584 
Epoch:  1   130000  /  371584 
Epoch:  1   140000  /  371584 
Epoch:  1   150000  /  371584 
Epoch:  1   160000  /  371584 
Epoch:  1   170000  /  371584 
Epoch:  1   180000  /  371584 
Epoch:  1   190000  /  371584 
Epoch:  1   200000  /  371584 
Epoch:  1   210000  /  371584 
Epoch:  1   220000  /  371584 
Epoch:  1   230000  /  371584 
Epoch:  1   240000  /  371584 
Epoch:  1   250000  /  371584 
Epoch:  1   260000  /  371584 
Epoch:  1   270000  /  371584 
Epoch:  1   280000  /  371584 
Epoch:  1   290000  /  371584 
Epoch:  1   300000  /  371584 
Epoch:

# Test models

In [0]:
!ls models

mod0.mod  mod1.mod  mod2.mod


In [42]:
import torch
import json
import numpy as np



modelPath = 'models/mod3.mod'
vocabPath = 'doc/vocab.json'
vocabHPath = 'doc/vocabH.json'
seqLen = 15

with open(vocabPath, 'r') as f:
    vocab = json.loads(f.read())
with open(vocabHPath, 'r') as f:
    vocabH = json.loads(f.read())

rev_vocab = {vocab[key]: key  for key in vocab }
rev_vocabh = {vocabH[key]: key  for key in vocabH }


def getHashtags(predict):
    pre = predict.detach().cpu().numpy()
    bestHashtags = pre.argsort()[-3:][::-1]
    for i in range(len(bestHashtags[0]) - 3, len(bestHashtags[0])):
      print('Preds ', ':', rev_vocabh[bestHashtags[0][i]])
    #return rev_vocabh[torch.argmax(predict,1).item()]
    
    
def transformText(text, seqLen, vocab):
    textInInt = np.zeros((1,seqLen))
    words = text.split()
    for i ,word in enumerate(words):
        if i <= len(words) - 1:
           textInInt[0][i] = vocab[word]
        else:
           textInInt[0][i] = vocab['<eos>']
    return torch.LongTensor(textInInt).cuda()

  
  
model = torch.load(modelPath)
for i in range(1000, 1100):
  textToPredict = ''

  for word in X_valid[i]:
    textToPredict += rev_vocab[word.item()] + ' '

  print('X:', textToPredict)

  print('Target:', rev_vocabh[Y_valid[i].item()])

  text_vec = transformText(textToPredict, seqLen, vocab)
  text_vec.reshape([1, seqLen])

  bo = model.forward(text_vec)

  getHashtags(bo)
  


X: . keeps it in the family with family farms! - <eos> <eos> <eos> <eos> <eos> 
Target: promotion
Preds  : exo
Preds  : music
Preds  : nowplaying
X: happy new year to you too <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> 
Target: happynewyear
Preds  : toronto
Preds  : delicious
Preds  : strangerthings
X: rt if you're w/ the challenger today at | live on ppv | <eos> <eos> 
Target: ufc219
Preds  : toronto
Preds  : retweet
Preds  : 
X: come hangout with me on <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> 
Target: bigolive
Preds  : np
Preds  : maga
Preds  : bigolive
X: it’s not anymore but we’re sending love <UNK> <eos> <eos> <eos> <eos> <eos> <eos> <eos> 
Target: ff
Preds  : mステにbts
Preds  : merrychristmaseveryone
Preds  : aldubsayolang
X: imo was a necessary moment for america. no <UNK> nation can tolerate <UNK> acting with 
Target: metoo
Preds  : iran
Preds  : ufc217
Preds  : fitness
X: good morning 😛 <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:67: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters().


 i entered a giveaway for a chance to win "black silicone oven <UNK> - 1 
Target: giveaway
Preds  : bitcoin
Preds  : win
Preds  : giveaway
X: 12/30 <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> mph sw <eos> <eos> <eos> <eos> <eos> 
Target: weather
Preds  : 
Preds  : iran
Preds  : ad
X: our was named by as one of <UNK> podcasts that will make your business more 
Target: podcast
Preds  : 
Preds  : iran
Preds  : nowplaying
X: . keeps it in the family with family farms! - <eos> <eos> <eos> <eos> <eos> 
Target: promotion
Preds  : exo
Preds  : music
Preds  : nowplaying
X: another just renewed their monthly subscription! see my exclusive content at <eos> <eos> <eos> <eos> 
Target: fan
Preds  : ps4live
Preds  : emabiggestfansjustinbieber
Preds  : fan
X: got my free chance to win <UNK> freecash in points on <UNK> tons of wins 
Target: instantwingame
Preds  : news
Preds  : ad
Preds  : instantwingame
X: follow everyone who rt this ✅ <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> 
Target: trapa

# Save all on drive

In [0]:
!mkdir drive/colab/test2 ; cp models/mod8.mod drive/colab/test2/bestMod.mod ; cp doc/* drive/colab/test2/ 

cp: cannot stat 'models/mod8.mod': No such file or directory
